# Cloning the repo

In [ ]:
# Clone the repo
!git clone https://github.com/sumuditha2001/plenoxels.git
%cd plenoxel

Cloning into 'plenoxel'...
remote: Enumerating objects: 1591, done.
remote: Counting objects: 100% (723/723), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 1591 (delta 661), reused 654 (delta 654), pack-reused 868 (from 1)
Receiving objects: 100% (1591/1591), 7.46 MiB | 28.62 MiB/s, done.
Resolving deltas: 100% (1182/1182), done.
/content/plenoxel


# Resolving Dependancies

In [ ]:


# Install PyTorch + dependencies
!pip install torch torchvision torchaudio
!pip install pybind11>=2.5.0 imageio imageio-ffmpeg matplotlib numpy scipy tqdm

# Optional: for dataset preprocessing
!pip install pycolmap

# Build and install svox2 (compiles the CUDA kernels)
!pip install -e . --verbose

!pip install lpips



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 68.2 MB/s eta 0:00:00
Using pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
Obtaining file:///content/plenoxel
  Running command python setup.py egg_info
  /content/plenoxel/setup.py:25: UserWarning: The environment variable `CUB_HOME` was not found.Installation will fail if your system CUDA toolkit version is less than 11.NVIDIA CUB can be downloaded from `https://github.com/NVIDIA/cub/releases`. You can unpack it to a location of your choice and set the environment variable `CUB_HOME` to the folder containing the `CMakeListst.txt` file.
    warnings.warn(
  /usr/local/lib/python3.12/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
  !!

          ********************************************************************************
          Requirements should be satisfied by a PEP 517 installer.
          If you are using pip, you can 

# Downloading dataset through kaggle

In [ ]:
# Install Kaggle API
!pip install kaggle

from google.colab import files
files.upload()  # Upload your kaggle.json here

# Set up Kaggle config with absolute path
!mkdir -p /root/.kaggle
!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

# Make sure target folder exists
!mkdir -p /content/plenoxel/data/nerf_synthetic

# Download the dataset using Kaggle API
!kaggle datasets download -d nguyenhung1903/nerf-synthetic-dataset -p /content/plenoxel/data/nerf_synthetic

# Unzip directly into the dataset folder
!unzip /content/plenoxel/data/nerf_synthetic/nerf-synthetic-dataset.zip -d /content/plenoxel/data/


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/nguyenhung1903/nerf-synthetic-dataset
License(s): unknown
100% 1.18G/1.18G [00:08<00:00, 149MB/s]
100% 1.18G/1.18G [00:08<00:00, 143MB/s]
Archive:  /content/plenoxel/data/nerf_synthetic/nerf-synthetic-dataset.zip
  inflating: /content/plenoxel/data/nerf_synthetic/README.txt  
  inflating: /content/plenoxel/data/nerf_synthetic/chair/test/r_0.png  
  inflating: /content/plenoxel/data/nerf_synthetic/chair/test/r_0_depth_0000.png  
  inflating: /content/plenoxel/data/nerf_synthetic/chair/test/r_1.png  
  inflating: /content/plenoxel/data/nerf_synthetic/chair/test/r_10.png  
  inflating: /content/plenoxel/data/nerf_synthetic/chair/test/r_100.png  
  inflating: /content/plenoxel/data/nerf_synthetic/chair/test/r_100_depth_0000.png  
  inflating: /content/plenoxel/data/nerf_synthetic/chair/test/r_101.png  
  inflating: /content/plenoxel/data/nerf_synthetic/chair/test/r_101_depth_0000.png  
  inflating: /content/plen

# Launching the experiment

In [ ]:
# Change directory to opt
%cd /content/plenoxel/opt

# Launch the training script with absolute paths
!bash /content/plenoxel/opt/launch.sh \
    lego_exp \
    0 \
    /content/plenoxel/data/nerf_synthetic/lego \
    -c /content/plenoxel/configs/syn.json


/content/plenoxel/opt
Launching experiment lego_exp
GPU 0
EXTRA /content/plenoxel/data/nerf_synthetic/lego -c /content/plenoxel/configs/syn.json
CKPT ckpt/lego_exp
LOGFILE ckpt/lego_exp/log
DETACH


# Running the training

In [ ]:
# Make sure you are in the opt folder
%cd /content/plenoxel/opt

# Make checkpoint directory
!mkdir -p /content/plenoxel/opt/ckpt/lego_exp

# Run the optimization script with absolute paths
!python /content/plenoxel/opt/opt.py \
    /content/plenoxel/data/nerf_synthetic/lego \
    -t /content/plenoxel/opt/ckpt/lego_exp \
    -c /content/plenoxel/opt/configs/syn.json \
    --use_cyclic_lr \
    --cyclic_base_lr 1e-5 \
    --cyclic_max_lr 1e-3 \
    --cyclic_step_size 4800 \
    --cyclic_mode exp_range \
    --cyclic_gamma 0.9998






/content/plenoxel/opt
2025-10-18 17:55:36.563406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760810136.583480   30297 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760810136.589566   30297 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760810136.606055   30297 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760810136.606088   30297 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760810136.606152   30297 computation_placer.cc:177] 

# Generating the video

In [ ]:
!python /content/plenoxel/opt/render_imgs.py \
    /content/plenoxel/opt/ckpt/lego_exp/ckpt.npz \
    /content/plenoxel/data/nerf_synthetic/lego \



In [ ]:
# !ffmpeg -r 30 -i /content/plenoxel/opt/ckpt/lego_exp/test_renders/%04d.png \
#        -c:v libx264 -pix_fmt yuv420p /content/lego_render.mp4
